In [54]:
import ast
import pandas as pd
import numpy as np

train_df = pd.read_csv("../../data/train-val-splits/qrels_args_docs_train_features_arg_ratio.tsv", sep="\t").dropna()
val_df = pd.read_csv("../../data/train-val-splits/qrels_args_docs_val_features_arg_ratio.tsv", sep="\t").dropna()
test_df = pd.read_csv("../../data/qrels_args_docs_test_features_arg_ratio.tsv", sep="\t").dropna()
#train_df['instr_mean'] = train_df['instr_mean'].apply(ast.literal_eval)
#val_df['instr_mean'] = val_df['instr_mean'].apply(ast.literal_eval)
#test_df['instr_mean'] = test_df['instr_mean'].apply(ast.literal_eval)
#print(len(train_df), len(val_df), len(test_df))

3007 691 500


In [55]:
# 'NAMEDENTITIES_NUM' not used: only zeros

feature_cols = ['LENGTH_OF_DOCUMENT', 'LENGTH_OF_PARAGRAPHS', 'LENGTH_OF_SENTENCES', 'LENGTH_OF_WORDS', 'STOPWORD_NUM', 'SPECCHARS_NUM', 
                'NUMERICS_NUM', 'FULLUPPERCASEWORD_NUM', 'UPPERCASEWORD_NUM', 'UPPERCASESENTENCE_NUM', 'VOCABULARY_RICHNESS',
                'VOCABULARY_RICHNESS_WITHOUT_STOPS', 'ACADEMIC_WORDS', 'PROFANITY_WORDS', 'TOPIC_KEYWORDS', 'ARGUMENTATIVE_WORDS', 
                'SUBJECTIVITY', 'LINKS_NUM', 'SENTIMENT', 'SPELLINGMISTAKES_NUM', 'READABILITY_flesch_kincaid', 'READABILITY_flesch', 
                'READABILITY_gunning_fog', 'READABILITY_coleman_liau', 'READABILITY_dale_chall', 'READABILITY_ari', 'READABILITY_linsear_write', 
                'READABILITY_spache', 'SENTENCE_TYPE_statement', 'SENTENCE_TYPE_question', 'SENTENCE_TYPE_exclamation', 'ARGUMENT RATIO']

train_df['feature_vector'] = train_df[feature_cols].agg(list, axis=1)
val_df['feature_vector'] = val_df[feature_cols].agg(list, axis=1)
test_df['feature_vector'] = test_df[feature_cols].agg(list, axis=1)
#print(len(train_df), len(val_df), len(test_df))

3007 691 500


In [56]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

scaler1 = MinMaxScaler(feature_range=(0, 1))
scaler2 = StandardScaler()

def norm_zero_one(l):
    return scaler1.fit_transform(l)

def standard_scale(l):
    return scaler2.fit_transform(l)

In [57]:
import tensorflow as tf

X_train, Y_train = list(), list()

#vector = train_df.feature_vector.tolist()
vector = norm_zero_one(train_df.feature_vector.tolist())
#vector = standard_scale(train_df.feature_vector.tolist())

for x in vector:
    x_train = np.array([np.array(val) for val in x])
    X_train.append(x_train)
    
for y in train_df.qual.tolist():
    y_train = np.array([np.array(val) for val in [y]])
    Y_train.append(y_train)

X_train = tf.cast(X_train , dtype=tf.float32)
Y_train = tf.cast(Y_train , dtype=tf.float32)

In [58]:
X_val, Y_val = list(), list()

#vector = val_df.feature_vector.tolist()
vector = norm_zero_one(val_df.feature_vector.tolist())
#vector = standard_scale(val_df.feature_vector.tolist())

for x in vector:
    x_val = np.array([np.array(val) for val in x])
    X_val.append(x_val)
    
for y in val_df.qual.tolist():
    y_val = np.array([np.array(val) for val in [y]])
    Y_val.append(y_val)

X_val = tf.cast(X_val , dtype=tf.float32)
Y_val = tf.cast(Y_val , dtype=tf.float32)

In [59]:
X_test = list()

#vector = test_df.feature_vector.tolist()
vector = norm_zero_one(test_df.feature_vector.tolist())
#vector = standard_scale(test_df.feature_vector.tolist())

for x in vector:
    x_test = np.array([np.array(val) for val in x])
    X_test.append(x_test)
    
X_test = tf.cast(X_test , dtype=tf.float32)

In [26]:
import pandas as pd
import re
import numpy as np

from sklearn.base import clone
from sklearn.metrics import classification_report, precision_score, recall_score
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.model_selection import StratifiedKFold

from transformers import pipeline, AutoTokenizer
from tqdm import tqdm

from matplotlib import pyplot as plt

import ast
import warnings

warnings.filterwarnings('ignore')

In [30]:
from numpy.random import seed
seed(42)

tf.random.set_seed(42)

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM
from sklearn.preprocessing import LabelBinarizer
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.metrics import TruePositives, Precision
import ast

# This neural network is used for the embeddings by the "large" transformer models

model2 = Sequential()
model2.add(Dense(512, input_shape=(32,), activation='relu', kernel_initializer='he_uniform'))
#model.add(Dropout(0.2))
model2.add(Dense(256, activation='relu'))
#model.add(Dropout(0.2))
model2.add(Dense(64, activation='relu'))
#model.add(Dropout(0.2))
model2.add(Dense(16, activation='relu'))
model2.add(Dense(3, activation='softmax'))
# compile the keras model
model2.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=["sparse_categorical_accuracy"])

In [196]:
#df_valid.qual.tolist()

In [237]:
#X_train[:20]

In [85]:
#tf.concat([X_train, X_val], axis=0).shape
#tf.concat([Y_train, Y_val], axis=0).shape

In [31]:
%%time 
#Train on train + val, predict on test

from keras.callbacks import EarlyStopping

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=50)

model2.fit(tf.concat([X_train, X_val], axis=0), tf.concat([Y_train, Y_val], axis=0), epochs=1000, batch_size=5, verbose=0, validation_split=0.2, callbacks=[es])
probs = model2.predict(X_test)
predictions = np.argmax(probs, axis=1)
#print(classification_report(y_true=Y_val, y_pred=predictions))

Epoch 53: early stopping
16/16 [==============================] - 0s 1ms/step
CPU times: user 2min 13s, sys: 7.74 s, total: 2min 20s
Wall time: 53.9 s


In [34]:
#probs

In [35]:
test_df['dnn_pred'] = predictions.tolist()
test_df['dnn_prob'] = probs.tolist()

### Logistic

In [37]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

clf = LogisticRegression(C=15, penalty="l2", solver="liblinear")

clf.fit(np.concatenate((X_train.numpy(), X_val.numpy()), axis=0), train_df.qual.tolist()+val_df.qual.tolist())
probs = clf.predict_proba(X_test.numpy())
predictions = clf.predict(X_test.numpy())

In [38]:
test_df['lr_pred'] = predictions.tolist()
test_df['lr_prob'] = probs.tolist()

### SVM

In [40]:
from sklearn.svm import SVC

clf = SVC(C=100, gamma=0.1, kernel='rbf', probability=True)

clf.fit(np.concatenate((X_train.numpy(), X_val.numpy()), axis=0), train_df.qual.tolist()+val_df.qual.tolist())
probs = clf.predict_proba(X_test.numpy())
predictions = clf.predict(X_test.numpy())

In [41]:
test_df['svm_pred'] = predictions.tolist()
test_df['svm_prob'] = probs.tolist()

In [131]:
#len(test_df)

### Random Forests

In [43]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(criterion='gini', max_depth=8, max_features='log2', n_estimators=200)

clf.fit(np.concatenate((X_train.numpy(), X_val.numpy()), axis=0), train_df.qual.tolist()+val_df.qual.tolist())
probs = clf.predict_proba(X_test.numpy())
predictions = clf.predict(X_test.numpy())

In [44]:
test_df['rf_pred'] = predictions.tolist()
test_df['rf_prob'] = probs.tolist()

In [142]:
#test_df.head()

### Naive Bayes

In [46]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

clf = MultinomialNB(alpha=0.01)

clf.fit(np.concatenate((X_train.numpy(), X_val.numpy()), axis=0), train_df.qual.tolist()+val_df.qual.tolist())
probs = clf.predict_proba(scaler.fit_transform(X_test.numpy()))
predictions = clf.predict(scaler.fit_transform(X_test.numpy()))

In [47]:
test_df['nb_pred'] = predictions.tolist()
test_df['nb_prob'] = probs.tolist()

### LightGBM

In [48]:
from lightgbm import LGBMClassifier

lgb = LGBMClassifier(
    max_depth=7,
    lambda_l1=0.1,
    lambda_l2=0.01,
    learning_rate=0.01,
    n_estimators=500,
    reg_aplha=1.1,
    colsample_bytree=0.9,
    subsample=0.9,
    n_jobs=5
)

In [49]:
lgb.fit(np.concatenate((X_train.numpy(), X_val.numpy()), axis=0), train_df.qual.tolist()+val_df.qual.tolist(), eval_set=[(X_val, Y_val)], eval_metric='auc_mu', verbose=False, early_stopping_rounds=50)

probs = lgb.predict_proba(X_test.numpy())
predictions = lgb.predict(X_test.numpy())

[LightGBM] [Warning] Unknown parameter: reg_aplha
[LightGBM] [Warning] lambda_l1 is set=0.1, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.1
[LightGBM] [Warning] lambda_l2 is set=0.01, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.01


In [50]:
test_df['lgb_pred'] = predictions.tolist()
test_df['lgb_prob'] = probs.tolist()

In [52]:
#test_df.tail()

In [53]:
test_df.to_csv("../../results-test/quality/chatnoir_10_custom_stopw_lemmas_features_predictions", sep="\t", index=False)